In [ ]:
#!pip install "adapter-transformers@git+https://github.com/akufeldt/adapter-transformers.git@debug#egg=adapter-transformers&subdirectory=adapter-transformers"

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Softmax

from typing import List, Optional, Tuple, Union, Dict, Any

from datasets import load_dataset, Dataset, DatasetDict, load_metric, load_from_disk, concatenate_datasets
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import PreTrainedModel, TrainingArguments, Trainer
#from transformers.adapters import AdapterTrainer

import pandas as pd
import numpy as np
import evaluate

import random
import math
import time
from tqdm import tqdm
import os
import json

/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
_numpy_rng = np.random.default_rng(seed)
random.seed(seed)
np.random.seed(seed)
torch.use_deterministic_algorithms(False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load in model

In [5]:
model_name = 'm2m100_418M'
experiment = 'en-ha_finetune_base_model-1'
dataset_name = 'data/en-ha'

In [6]:
model = M2M100ForConditionalGeneration.from_pretrained(f"facebook/{model_name}")
# model = torch.nn.DataParallel(model, device_ids=[2, 3, 4])
model = model.to(device)
tokenizer = M2M100Tokenizer.from_pretrained(f"facebook/{model_name}")

# Prepare data

In [7]:
src_lang = 'en'
tgt_lang = 'ha'
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "ha"

In [8]:
# Load the datasets from Hugging Face Hub
original_train_dataset = load_dataset("pranjali97/ha-en_RL-grow1_train", split='train')
original_valid_dataset = load_dataset("pranjali97/ha-en_RL-grow1_valid", split='train')  # Assuming the split is also 'train'


In [9]:
# Filter the datasets to only include samples with a score > 0.6
filtered_train_dataset = original_train_dataset.filter(lambda example: example['score'] > 0.6)
filtered_valid_dataset = original_valid_dataset.filter(lambda example: example['score'] > 0.6)

In [10]:
filtered_train_dataset = filtered_train_dataset.remove_columns(['score', 'ref'])
filtered_valid_dataset = filtered_valid_dataset.remove_columns(['score', 'ref'])

In [11]:
train_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/cleaned_train.csv', split='train')
valid_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/cleaned_dev.csv', split='train')

In [12]:
train_dataset

Dataset({
    features: ['en', 'ha'],
    num_rows: 9818
})

In [13]:
train_dataset = train_dataset.rename_column('ha', 'src')
valid_dataset = valid_dataset.rename_column('ha', 'src')
train_dataset = train_dataset.rename_column('en', 'mt')
valid_dataset = valid_dataset.rename_column('en', 'mt')



In [14]:
train_dataset

Dataset({
    features: ['mt', 'src'],
    num_rows: 9818
})

In [15]:
train_dataset = concatenate_datasets([train_dataset, filtered_train_dataset])
valid_dataset = concatenate_datasets([valid_dataset, filtered_valid_dataset])  
train_dataset = train_dataset.shuffle(seed=42)
valid_dataset = valid_dataset.shuffle(seed=42)

/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [16]:
# Define the preprocess function
def preprocess_function(examples):
    inputs = examples['src']  # Hausa sentences
    targets = examples['mt']  # English translations
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    # Return the tokenized inputs and labels
    return {'input_ids': model_inputs['input_ids'], 'attention_mask': model_inputs['attention_mask'], 'labels': labels['input_ids']}

# Apply the preprocess function to the datasets
tokenized_train_dataset = train_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)
tokenized_valid_dataset = valid_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

# Create the DatasetDict
tokenized_dataset = DatasetDict({
    'train': tokenized_train_dataset,  # Directly assign the processed dataset
    'validation': tokenized_valid_dataset  # Directly assign the processed dataset
})

Map:   0%|          | 0/12638 [00:00<?, ? examples/s]/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1449/1449 [00:00<00:00, 3257.23 examples/s]


In [17]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 12638
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1449
    })
})

# not run

In [ ]:
dataset = DatasetDict({'train':Dataset.from_pandas(pd.read_csv(f'{dataset_name}/cleaned_train.csv')).shuffle(seed=seed),
                        'validation':Dataset.from_pandas(pd.read_csv(f'{dataset_name}/cleaned_dev.csv')).shuffle(seed=seed),
                        'test':Dataset.from_pandas(pd.read_csv(f'{dataset_name}/test.csv')).shuffle(seed=seed),
})

In [ ]:
dataset['test'] = dataset['test'].rename_column('sentence_eng_Latn','en')
dataset['test'] = dataset['test'].rename_column('sentence_hau_Latn','ha')

In [ ]:
dataset

In [ ]:
def preprocess_function(examples):
    inputs = [example for example in examples[src_lang]]
    targets = [example for example in examples[tgt_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=256, truncation=True, padding="max_length")
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names['train'])

In [ ]:
tokenized_dataset

# Training Setup

In [18]:
sacrebleu = evaluate.load("sacrebleu")
wer = evaluate.load("wer")

In [19]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    pred_ids = eval_preds.predictions
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    preds = np.argmax(pred_ids, axis=-1)

    # removeme
    #import warnings
    #warnings.warn(f"unprocessed preds: {preds[0]}\n)")
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) 

    # removeme
    #warnings.warn(f"unprocessed decoded labels: {tokenizer.batch_decode(labels)[0]}\n)")

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # remove me
    #inputs = eval_preds.input_ids
    #decoded_inputs = tokenizer.batch_decode(inputs)
    
    # Removeme
    import warnings
    warnings.warn(f"preds: {decoded_preds[0]}\n)")
    warnings.warn(f"labels: {decoded_labels[0]}\n)")

    bleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    metrics = {"bleu": bleu_result["score"]}

    flattened_decoded_labels = [' '.join([str(x) for x in l]) for l in decoded_labels]
    wer_score = wer.compute(predictions=decoded_preds, references=flattened_decoded_labels)
    metrics["wer"] = wer_score

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    metrics["gen_len"] = np.mean(prediction_lens)
    metrics = {k: round(v, 4) for k, v in metrics.items()}
    return metrics


In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [21]:
training_args = TrainingArguments(
    f"./lang_adapters/{experiment}/model",
    # evaluation_strategy="steps",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    warmup_steps=0,
    # lr_scheduler_type='cosine_with_restarts',
    # gradient_accumulation_steps=4,
    eval_accumulation_steps=16,
    # gradient_checkpointing=True,
    # predict_with_generate=True,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    # eval_steps=5,
    save_strategy="epoch",
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Wer,Gen Len
1,0.249000,0.263111,25.268500,0.994100,33.409200
2,0.257900,0.241613,28.138300,0.994600,33.409200
3,0.165000,0.234886,29.427100,0.995100,33.409200
4,0.147600,0.232156,30.590900,0.995500,33.409200
5,0.132200,0.233843,31.648400,0.995200,33.409200
6,0.110000,0.236766,32.067500,0.995000,33.409200
7,0.106700,0.239103,32.442500,0.994600,33.409200
8,0.072100,0.244547,32.968200,0.995000,33.409200
9,0.057000,0.248213,32.886000,0.995100,33.409200
10,0.061300,0.250425,32.859900,0.994800,33.409200


/tmp/ipykernel_22620/3289696467.py:37: UserWarning: preds: God is: Johnyah 1:2,14; Romans 9;5; Colossians 2:9; Hebrews 1:8; 1 Yahweh 5:20.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_22620/3289696467.py:38: UserWarning: labels: God is: Yahweh 1:2,14; Romans 9;5; Colossians 2:9; Hebrews 1:8; 1 Yahweh 5:20.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_22620/3289696467.py:37: UserWarning: preds: God is: Johnyah 1:2, 14; Romans 9;5; Colossians 2:9; Hebrews 1:8; 1 Yahweh 5:20.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_22620/3289696467.py:38: UserWarning: labels: God is: Yahweh 1:2,14; Romans 9;5; Colossians 2:9; Hebrews 1:8; 1 Yahweh 5:20.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_22620/3289696467.py:37: UserWarning: preds: God is: Johnyah 1:2,14; Romans 9;5; Colossians 2:9; Hebrews 1:8; 1 Yahweh 5:20.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_22620/3289696467.py:38: UserWa

TrainOutput(global_step=5925, training_loss=0.17147542449492442, metrics={'train_runtime': 5253.6947, 'train_samples_per_second': 36.083, 'train_steps_per_second': 1.128, 'total_flos': 1.0270450485559296e+17, 'train_loss': 0.17147542449492442, 'epoch': 15.0})

In [23]:
base_model_path = f'./0.6_base_model/{experiment}'


if not os.path.exists(base_model_path):
    os.makedirs(base_model_path)

trainer.save_model(base_model_path)

In [24]:
#use the saved model for evaluation
#base_model_path = f'./0.6_base_model/{experiment}'
#model = M2M100ForConditionalGeneration.from_pretrained(base_model_path)
#model = model.to(device)
test_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/test.csv', split='train')

#rename 'sentence_eng_Latn' to 'en' and 'sentence_hau_Latn' to 'ha'
test_dataset = test_dataset.rename_column('sentence_eng_Latn','mt')
test_dataset = test_dataset.rename_column('sentence_hau_Latn','src')

#tokenize the dataset
tokenized_test_dataset = test_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

#convert the dataset into dataset dict
tokenized_test_dataset = DatasetDict({
    'test': tokenized_test_dataset  # Directly assign the processed dataset
})

eval_results = trainer.evaluate(tokenized_test_dataset["test"])

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1012/1012 [00:00<00:00, 2982.25 examples/s]


/tmp/ipykernel_22620/3289696467.py:37: UserWarning: preds: “We have have fourmonth liveslong childrenirt un is not-smabetical and are to have withabetic," I heard.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_22620/3289696467.py:38: UserWarning: labels: "We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")


In [25]:
eval_results

{'eval_loss': 0.4250865578651428,
 'eval_bleu': 15.0956,
 'eval_wer': 0.9945,
 'eval_gen_len': 34.2757,
 'eval_runtime': 113.7833,
 'eval_samples_per_second': 8.894,
 'eval_steps_per_second': 1.116,
 'epoch': 15.0}

In [42]:
# Save model
if not os.path.exists(f'./base_model/{experiment}'):
    os.mkdir(f'/base_model/{experiment}')
    
trainer.save_model(f"./base_model/{experiment}")

FileNotFoundError: [Errno 2] No such file or directory: '/base_model/en-ha_finetune_base_model-1'

# Eval finetuned model on test set

In [26]:
# Evaluate performance
src_lang = 'en'
tgt_lang = 'ha'
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "ha"

In [27]:
model = M2M100ForConditionalGeneration.from_pretrained(f"./base_model/{experiment}")
# model = torch.nn.DataParallel(model, device_ids=[2, 3, 4])
model = model.to(device)
tokenizer = M2M100Tokenizer.from_pretrained(f"./base_model/{experiment}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
test_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/test.csv', split='train')

In [52]:
test_dataset

Dataset({
    features: ['sentence_eng_Latn', 'sentence_hau_Latn'],
    num_rows: 1012
})

In [29]:
#rename 'sentence_eng_Latn' to 'en' and 'sentence_hau_Latn' to 'ha'
test_dataset = test_dataset.rename_column('sentence_eng_Latn','mt')
test_dataset = test_dataset.rename_column('sentence_hau_Latn','src')

In [30]:
#tokenize the dataset
tokenized_test_dataset = test_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1012/1012 [00:00<00:00, 3020.01 examples/s]


In [31]:
#convert the dataset into dataset dict
tokenized_test_dataset = DatasetDict({
    'test': tokenized_test_dataset  # Directly assign the processed dataset
})

In [32]:
tokenized_test_dataset

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1012
    })
})

In [33]:
#run evaluation
eval_results = trainer.evaluate(tokenized_test_dataset["test"])

KeyboardInterrupt: 

In [ ]:
eval_results

{'eval_loss': 0.46018558740615845,
 'eval_bleu': 14.5322,
 'eval_wer': 0.9948,
 'eval_gen_len': 34.2757,
 'eval_runtime': 174.9085,
 'eval_samples_per_second': 5.786,
 'eval_steps_per_second': 1.446,
 'epoch': 15.0}

# 0.7 Filtered Data

In [7]:
src_lang = 'en'
tgt_lang = 'ha'
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "ha"

In [21]:
# Load the datasets from Hugging Face Hub
original_train_dataset = load_dataset("pranjali97/ha-en_RL-grow1_train", split='train')
original_valid_dataset = load_dataset("pranjali97/ha-en_RL-grow1_valid", split='train')

new_filtered_train_dataset = original_train_dataset.filter(lambda example: example['score'] > 0.7)
new_filtered_valid_dataset = original_valid_dataset.filter(lambda example: example['score'] > 0.7)
new_filtered_train_dataset = new_filtered_train_dataset.remove_columns(['score', 'ref'])
new_filtered_valid_dataset = new_filtered_valid_dataset.remove_columns(['score', 'ref'])

train_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/cleaned_train.csv', split='train')
valid_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/cleaned_dev.csv', split='train')

train_dataset = train_dataset.rename_column('ha', 'src')
valid_dataset = valid_dataset.rename_column('ha', 'src')
train_dataset = train_dataset.rename_column('en', 'mt')
valid_dataset = valid_dataset.rename_column('en', 'mt')

new_train_dataset = concatenate_datasets([train_dataset, new_filtered_train_dataset])
new_valid_dataset = concatenate_datasets([valid_dataset, new_filtered_valid_dataset])  
new_train_dataset = new_train_dataset.shuffle(seed=42)
new_valid_dataset = new_valid_dataset.shuffle(seed=42)

/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [22]:
train_dataset

Dataset({
    features: ['mt', 'src'],
    num_rows: 9818
})

In [23]:
new_filtered_train_dataset

Dataset({
    features: ['src', 'mt'],
    num_rows: 802
})

In [24]:
new_train_dataset

Dataset({
    features: ['mt', 'src'],
    num_rows: 10620
})

In [25]:
# Define the preprocess function
def preprocess_function(examples):
    inputs = examples['src']  # Hausa sentences
    targets = examples['mt']  # English translations
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    # Return the tokenized inputs and labels
    return {'input_ids': model_inputs['input_ids'], 'attention_mask': model_inputs['attention_mask'], 'labels': labels['input_ids']}

# Apply the preprocess function to the datasets
new_tokenized_train_dataset = new_train_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)
new_tokenized_valid_dataset = new_valid_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

# Create the DatasetDict
new_tokenized_dataset = DatasetDict({
    'train': new_tokenized_train_dataset,  # Directly assign the processed dataset
    'validation': new_tokenized_valid_dataset  # Directly assign the processed dataset
})

Map:   0%|          | 0/10620 [00:00<?, ? examples/s]/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1199/1199 [00:00<00:00, 3336.15 examples/s]


In [26]:
new_tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10620
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1199
    })
})

In [27]:
sacrebleu = evaluate.load("sacrebleu")
wer = evaluate.load("wer")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    pred_ids = eval_preds.predictions
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    preds = np.argmax(pred_ids, axis=-1)

    # removeme
    #import warnings
    #warnings.warn(f"unprocessed preds: {preds[0]}\n)")
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) 

    # removeme
    #warnings.warn(f"unprocessed decoded labels: {tokenizer.batch_decode(labels)[0]}\n)")

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # remove me
    #inputs = eval_preds.input_ids
    #decoded_inputs = tokenizer.batch_decode(inputs)
    
    # Removeme
    import warnings
    warnings.warn(f"preds: {decoded_preds[0]}\n)")
    warnings.warn(f"labels: {decoded_labels[0]}\n)")

    bleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    metrics = {"bleu": bleu_result["score"]}

    flattened_decoded_labels = [' '.join([str(x) for x in l]) for l in decoded_labels]
    wer_score = wer.compute(predictions=decoded_preds, references=flattened_decoded_labels)
    metrics["wer"] = wer_score

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    metrics["gen_len"] = np.mean(prediction_lens)
    metrics = {k: round(v, 4) for k, v in metrics.items()}
    return metrics



In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [29]:
training_args = TrainingArguments(
    f"./lang_adapters/{experiment}/model",
    # evaluation_strategy="steps",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    warmup_steps=0,
    # lr_scheduler_type='cosine_with_restarts',
    # gradient_accumulation_steps=4,
    eval_accumulation_steps=16,
    # gradient_checkpointing=True,
    # predict_with_generate=True,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    # eval_steps=5,
    save_strategy="epoch",
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=new_tokenized_dataset["train"],
    eval_dataset=new_tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Wer,Gen Len
1,0.238500,0.258932,27.087200,0.994900,34.075100
2,0.220800,0.247632,28.919900,0.994400,34.075100
3,0.182100,0.243424,29.627800,0.994800,34.075100
4,0.130800,0.244643,30.753300,0.994600,34.075100
5,0.119900,0.246674,31.211700,0.994800,34.075100
6,0.102700,0.250211,31.893600,0.994400,34.075100
7,0.106800,0.254414,31.998400,0.994800,34.075100
8,0.069600,0.259981,32.144800,0.994800,34.075100
9,0.076000,0.267587,32.227300,0.995300,34.075100
10,0.056500,0.269285,32.774900,0.994500,34.075100


/tmp/ipykernel_17191/104191538.py:40: UserWarning: preds: I would like to tell this opportunity to tell that fact that I province is the theited by Sh Shia and Sunni.s.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_17191/104191538.py:41: UserWarning: labels: I would like to take this opportunity to mention a point. Your province is inhabited by both Shia and Sunni Muslims.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_17191/104191538.py:40: UserWarning: preds: I would like to tell this opportunity to tell that few that You province is where theited by Sh Shia and Sunni peoples.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_17191/104191538.py:41: UserWarning: labels: I would like to take this opportunity to mention a point. Your province is inhabited by both Shia and Sunni Muslims.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_17191/104191538.py:40: UserWarning: preds: I would like to tell this opportuni

TrainOutput(global_step=4980, training_loss=0.09708952237085167, metrics={'train_runtime': 4056.3316, 'train_samples_per_second': 39.272, 'train_steps_per_second': 1.228, 'total_flos': 8.63049407791104e+16, 'train_loss': 0.09708952237085167, 'epoch': 15.0})

In [31]:
base_model_path = f'./0.7_base_model/{experiment}'

if not os.path.exists(base_model_path):
    os.makedirs(base_model_path)

trainer.save_model(base_model_path)

In [32]:
test_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/test.csv', split='train')

#rename 'sentence_eng_Latn' to 'en' and 'sentence_hau_Latn' to 'ha'
test_dataset = test_dataset.rename_column('sentence_eng_Latn','mt')
test_dataset = test_dataset.rename_column('sentence_hau_Latn','src')

#tokenize the dataset
tokenized_test_dataset = test_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

#convert the dataset into dataset dict
tokenized_test_dataset = DatasetDict({
    'test': tokenized_test_dataset  # Directly assign the processed dataset
})

new_eval_results = trainer.evaluate(tokenized_test_dataset["test"])

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1012/1012 [00:00<00:00, 3057.94 examples/s]


/tmp/ipykernel_17191/104191538.py:40: UserWarning: preds: OnceWe are are fourmonth hardlong couock- has not-adabetical and have to have thabetic," I is.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_17191/104191538.py:41: UserWarning: labels: "We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")


In [33]:
new_eval_results

{'eval_loss': 0.42892614006996155,
 'eval_bleu': 15.2755,
 'eval_wer': 0.9946,
 'eval_gen_len': 34.2757,
 'eval_runtime': 110.6888,
 'eval_samples_per_second': 9.143,
 'eval_steps_per_second': 1.147,
 'epoch': 15.0}

# Training with Original Data

In [7]:
src_lang = 'en'
tgt_lang = 'ha'
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "ha"

In [8]:
train_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/cleaned_train.csv', split='train')
valid_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/cleaned_dev.csv', split='train')

train_dataset = train_dataset.rename_column('ha', 'src')
valid_dataset = valid_dataset.rename_column('ha', 'src')
train_dataset = train_dataset.rename_column('en', 'mt')
valid_dataset = valid_dataset.rename_column('en', 'mt')



In [9]:
train_dataset

Dataset({
    features: ['mt', 'src'],
    num_rows: 9818
})

In [10]:
# Define the preprocess function
def preprocess_function(examples):
    inputs = examples['src']  # Hausa sentences
    targets = examples['mt']  # English translations
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    # Return the tokenized inputs and labels
    return {'input_ids': model_inputs['input_ids'], 'attention_mask': model_inputs['attention_mask'], 'labels': labels['input_ids']}

# Apply the preprocess function to the datasets
og_tokenized_train_dataset = train_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)
og_tokenized_valid_dataset = valid_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

# Create the DatasetDict
og_tokenized_dataset = DatasetDict({
    'train': og_tokenized_train_dataset,  # Directly assign the processed dataset
    'validation': og_tokenized_valid_dataset  # Directly assign the processed dataset
})

Map:   0%|          | 0/9818 [00:00<?, ? examples/s]/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1113/1113 [00:00<00:00, 3206.46 examples/s]


In [11]:
og_tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9818
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1113
    })
})

In [12]:
sacrebleu = evaluate.load("sacrebleu")
wer = evaluate.load("wer")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    pred_ids = eval_preds.predictions
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    preds = np.argmax(pred_ids, axis=-1)

    # removeme
    #import warnings
    #warnings.warn(f"unprocessed preds: {preds[0]}\n)")
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) 

    # removeme
    #warnings.warn(f"unprocessed decoded labels: {tokenizer.batch_decode(labels)[0]}\n)")

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # remove me
    #inputs = eval_preds.input_ids
    #decoded_inputs = tokenizer.batch_decode(inputs)
    
    # Removeme
    import warnings
    warnings.warn(f"preds: {decoded_preds[0]}\n)")
    warnings.warn(f"labels: {decoded_labels[0]}\n)")

    bleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    metrics = {"bleu": bleu_result["score"]}

    flattened_decoded_labels = [' '.join([str(x) for x in l]) for l in decoded_labels]
    wer_score = wer.compute(predictions=decoded_preds, references=flattened_decoded_labels)
    metrics["wer"] = wer_score

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    metrics["gen_len"] = np.mean(prediction_lens)
    metrics = {k: round(v, 4) for k, v in metrics.items()}
    return metrics



In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [14]:
training_args = TrainingArguments(
    f"./lang_adapters/{experiment}/model",
    # evaluation_strategy="steps",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    warmup_steps=0,
    # lr_scheduler_type='cosine_with_restarts',
    # gradient_accumulation_steps=4,
    eval_accumulation_steps=16,
    # gradient_checkpointing=True,
    # predict_with_generate=True,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    # eval_steps=5,
    save_strategy="epoch",
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=og_tokenized_dataset["train"],
    eval_dataset=og_tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Wer,Gen Len
1,0.325300,0.297524,23.138400,0.996300,35.245300
2,0.290600,0.268421,26.638400,0.995000,35.245300
3,0.234000,0.257582,28.859200,0.995400,35.245300
4,0.197300,0.254482,29.713700,0.995600,35.245300
5,0.143500,0.255224,30.268500,0.994600,35.245300
6,0.141500,0.256460,30.893600,0.994900,35.245300
7,0.106800,0.261120,31.531100,0.994500,35.245300
8,0.079300,0.265503,31.701200,0.994700,35.245300
9,0.081800,0.270239,31.923900,0.994500,35.245300
10,0.076400,0.274925,32.194800,0.995000,35.245300


/tmp/ipykernel_17580/104191538.py:40: UserWarning: preds: Today in our neighic provinaries there there are students schij scholars and are proud of this the of Basij.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_17580/104191538.py:41: UserWarning: labels: Today in our Islamic seminaries, there are great basiji scholars who are proud of being members of Basij.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_17580/104191538.py:40: UserWarning: preds: Today there our hallic seminaries are there are high schiji scholars and are proud of this the of Basij.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_17580/104191538.py:41: UserWarning: labels: Today in our Islamic seminaries, there are great basiji scholars who are proud of being members of Basij.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_17580/104191538.py:40: UserWarning: preds: Today there our schoolsic seminaries, there are also schiji scholars and a

TrainOutput(global_step=4605, training_loss=0.20949620604838662, metrics={'train_runtime': 3589.6463, 'train_samples_per_second': 41.026, 'train_steps_per_second': 1.283, 'total_flos': 7.978737368825856e+16, 'train_loss': 0.20949620604838662, 'epoch': 15.0})

In [16]:
trained_model_path = f'./og_base_model/{experiment}'

if not os.path.exists(trained_model_path):
    os.makedirs(trained_model_path)

trainer.save_model(trained_model_path)

In [17]:
test_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/test.csv', split='train')

#rename 'sentence_eng_Latn' to 'en' and 'sentence_hau_Latn' to 'ha'
test_dataset = test_dataset.rename_column('sentence_eng_Latn','mt')
test_dataset = test_dataset.rename_column('sentence_hau_Latn','src')

#tokenize the dataset
tokenized_test_dataset = test_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

#convert the dataset into dataset dict
tokenized_test_dataset = DatasetDict({
    'test': tokenized_test_dataset  # Directly assign the processed dataset
})

new_eval_results = trainer.evaluate(tokenized_test_dataset["test"])

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1012/1012 [00:00<00:00, 3043.94 examples/s]


/tmp/ipykernel_17580/104191538.py:40: UserWarning: preds: “We are have 4month longlong peopleock- is without-sabetical and have to have thabetic," I says.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_17580/104191538.py:41: UserWarning: labels: "We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")


In [18]:
new_eval_results

{'eval_loss': 0.41901615262031555,
 'eval_bleu': 15.2866,
 'eval_wer': 0.9947,
 'eval_gen_len': 34.2757,
 'eval_runtime': 118.8275,
 'eval_samples_per_second': 8.517,
 'eval_steps_per_second': 1.069,
 'epoch': 15.0}

# 0.8 Threshold